# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="readonly/Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="readonly/Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [8]:
# Use the following data for this assignment:

import pandas as pd
import numpy as np

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])

In [9]:
%matplotlib widget
import matplotlib.pyplot as plt
import scipy.stats as st
import matplotlib.transforms as transforms
from matplotlib import cm, colors
from itertools import chain

In [10]:
# Find deltas between mean and 95% confidence interval for each distribution
def find_interval_size(row):
    low, high = st.norm.interval(alpha=0.95, loc=np.mean(row), scale=st.sem(row))
    return (high - low)/2

interval_sizes = df.apply(find_interval_size, axis=1)

# Find lower and upper 95% confidence bounds for each distribution
def find_conf_bounds(row):
    return st.norm.interval(alpha=0.95, loc=np.mean(row), scale=st.sem(row))

conf_bounds = df.apply(find_conf_bounds, axis=1)

# Find means and standard errors for each distribution
means = df.mean(axis=1) 
ses = df.std(axis=1)/np.sqrt(df.shape[1])

In [11]:
# Set a starting y value

y = int(input())

 42000


In [12]:
# Create normalizer to normalize z scores in 95% CI, then get the colormap using a default option
normalizer = colors.Normalize(vmin=-1.96, vmax=1.96, clip=True)
cmap = cm.get_cmap('coolwarm')

# Create empty colors_df to house normalized values and the RGBA values
colors_df = pd.DataFrame([])

colors_df['normalized_value'] = normalizer((means - y) / ses)
colors_df['colors'] = colors_df.apply(lambda x: list(chain.from_iterable(cmap(x))), axis=1)

In [13]:
# Plot figure and make pretty
fig, ax = plt.subplots()
bars = plt.bar(x=[0.5, 1.5, 2.5, 3.5], height=means, 
               color=colors_df['colors'], alpha=0.8, 
               yerr=interval_sizes, capsize=10, align='center', width=1)

xy_axes = plt.axis([0, 6, 0, 55000])
x_labels = plt.xticks(ticks=[0.5, 1.5, 2.5, 3.5], labels=df.index)
y_line = plt.axhline(y=y, xmax=0.8, color='red', alpha=1, linewidth=1)
big_title = plt.suptitle('Which Distribution is Most Representative of Sample y={}?'.format(y))
small_title = plt.title('Bar height is the mean and whiskers show 95% CI', fontsize=10)

# Put y value text on the plot
trans = transforms.blended_transform_factory(ax.get_yticklabels()[0].get_transform(), 
                                             ax.transData)
y_text = ax.text(0.92,y, "{:.0f}".format(y), color="red", transform=trans, 
        ha="right", va="center")

# Make a colorbar
my_bar = plt.colorbar(cm.ScalarMappable(norm=colors.Normalize(), cmap=cmap), orientation='horizontal', 
             shrink=0.8, pad=0.1, aspect=40)

# Caption the colorbar
caption = 'Color indicates normalized value of y on the CI\nBluer: y is ABOVE CI\nRedder: y is BELOW CI'.format(y)
bar_caption = fig.text(.5, .04, caption, ha='center')

# Remove top and left borders
for side in ['top', 'right']:
    ax.spines[side].set_visible(False)

# Create buttonclick event
def onclick(event):
    for i, bar in enumerate(bars):
        shade = cmap(normalizer((means.iloc[i] - event.ydata) / ses.iloc[i]))
        bar.set_color(shade) 
    y_line.set_ydata(event.ydata)
    y_text.set_text('{:.0f}'.format(event.ydata))
    y_text.set_position((0.92, event.ydata))

# Connect the button press event
fig.canvas.mpl_connect('button_press_event', onclick);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Bonus: Getting normalized values manually.
colors_df_manual = pd.DataFrame([])

colors_df_manual['z_scores'] = (means - y) / ses
colors_df_manual['norms'] = (colors_df_manual['z_scores'] - (-1.96)) / (1.96 - (-1.96))
colors_df_manual['rgba'] = colors_df_manual.apply(lambda x: list(chain.from_iterable(cmap(x))), axis=1)

In [17]:
# Bonus: What would be the error from assuming linear proportionality instead of using p-values to normalize y on the confidence interval?
# This seemed like a too easy trap to fall into
def get_norms(y, low, high):
    if y <= low:
        return 1-1e-10
    elif y >= high:
        return 0+1e-10
    else:
        return 1 - (y-low)/(high-low)
    
def get_norms_p(y, low, high, m, s):
    def get_p(x, m, s):
        z = (x - m)/s
        return st.norm.cdf(z)

    if y <= low:
        return 1-1e-10
    elif y >= high:
        return 0+1e-10
    else:
        p_y = get_p(y, m, s)
        p_low = get_p(low, m, s)
        p_high = get_p(high, m, s)
        return 1 - (p_y - p_low) / (p_high - p_low)

# Normalized values assuming linear proportions    
norms = [get_norms(y, conf_bounds.iloc[i][0], conf_bounds.iloc[i][1]) for i in range(4)]

# Normalized values using true distribution
norms_p = [get_norms_p(y, conf_bounds.iloc[i][0], conf_bounds.iloc[i][1],
                        means.iloc[i], ses.iloc[i]) for i in range(4)]

# Get the error for each bar
errors = [abs(norms[i] - norms_p[i]) for i in range(4)]
print('The errors from assuming linear proportionality are:')
for i, e in enumerate(errors):
    print('Bar {}: ~{:.8f}'.format(i+1, e))

The errors from assuming linear proportionality are:
Bar 1: ~0.00000000
Bar 2: ~0.01393783
Bar 3: ~0.10338030
Bar 4: ~0.00000000


In [259]:
# plt.savefig(fname='harder_option_plot.png')